### Testing New csv Load In Method

In [ ]:
# Must Pip Install python-decouple each time. 
#!pip install python-decouple

In [5]:
# Testing a Progress Bar Install for the Vectorizer. 
#import pyprind
#import psutil
# Importing TimeIt to view execution time
import timeit

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
#import theano
import spacy
import s3
nlp = spacy.load('en')

In [2]:
# Everytime you enter
bucket = s3.Bucket('yelpchallenge1')

In [34]:
# Look inside the bucket.
bucket.contents

['datasets/',
 'datasets/df.csv',
 'notebooks/',
 'notebooks/data_cleanup.ipynb',
 'notebooks/vectorization_exploration.ipynb',
 'notebooks/yelp_data_initial_exploration.ipynb',
 'vect_1.sav']

In [4]:
# Only have to run this once.
# Installs the .csv 'Locally' on SageMaker Instance

#bucket.get('datasets/df.csv', 'df.csv')

In [6]:
# Read df.csv
# Veiwing Execution Time. 
#%timeit 
df = pd.read_csv('df.csv')

/home/ec2-user/anaconda3/envs/ydc1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,1,2,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# check import
df.head()

,Unnamed: 0,stars,text,date,total_votes,tokens
0,0,1,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7.0,"['total', 'horrible', 'service', 'crooks', 'ac..."
1,1,5,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0.0,"['adore', 'travis', 'hard', 'rock', 'kelly', '..."
2,2,5,I have to say that this office really has it t...,2016-11-09 20:09:03,3.0,"['office', 'organized', 'friendly', 'phillipp'..."
3,3,5,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0.0,"['went', 'lunch', 'steak', 'sandwich', 'delici..."
4,4,1,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7.0,"['today', 'second', 'sessions', 'paid', 'sessi..."


In [8]:
# check datatypes of columns in dataframe
df.dtypes

Unnamed: 0      object
stars           object
text            object
date            object
total_votes    float64
tokens          object
dtype: object

In [9]:
# drop unwanted columns
df = df.drop(columns=['Unnamed: 0'])

In [11]:
df.head()

,stars,text,date,total_votes,tokens
0,1,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7.0,"['total', 'horrible', 'service', 'crooks', 'ac..."
1,5,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0.0,"['adore', 'travis', 'hard', 'rock', 'kelly', '..."
2,5,I have to say that this office really has it t...,2016-11-09 20:09:03,3.0,"['office', 'organized', 'friendly', 'phillipp'..."
3,5,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0.0,"['went', 'lunch', 'steak', 'sandwich', 'delici..."
4,1,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7.0,"['today', 'second', 'sessions', 'paid', 'sessi..."


In [12]:
df.dtypes

stars           object
text            object
date            object
total_votes    float64
tokens          object
dtype: object

In [11]:
df.shape

(6876898, 5)

In [27]:
# create mini-dataframe for testing
# want to make sure works locally on small dataset before scaling to entire dataset/AWS
mini_df = df.head(10)

In [15]:
# exploring TFIDF Vectorizer

In [29]:
# new tokenizer
 def tokenize(document):
    doc = nlp(document)
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [30]:
# create variable to feed into TFIDF Vectorizer fit_transform
# to be updated to 'text' column of main dataframe (df['text']) for vectorization in AWS
data = mini_df['text']

In [31]:
data.head()

0    Total bill for this horrible service? Over $8G...
1    I *adore* Travis at the Hard Rock's new Kelly ...
2    I have to say that this office really has it t...
3    Went in for a lunch. Steak sandwich was delici...
4    Today was my second out of three sessions I ha...
Name: text, dtype: object

In [32]:
data.shape

(10,)

In [33]:
#Checking for missing values/ Before Drop.nas
data.isna().sum()

0

In [16]:
data.isna().any()

True

In [17]:
# Dropping na from Data
data = data.dropna()

In [18]:
#BEGONE NON-EXISTENT DATA
data.isna().sum()

0

In [19]:
data[2]

"I have to say that this office really has it together, they are so organized and friendly!  Dr. J. Phillipp is a great dentist, very friendly and professional.  The dental assistants that helped in my procedure were amazing, Jewel and Bailey helped me to feel comfortable!  I don't have dental insurance, but they have this insurance through their office you can purchase for $80 something a year and this gave me 25% off all of my dental work, plus they helped me get signed up for care credit which I knew nothing about before this visit!  I highly recommend this office for the nice synergy the whole office has!"

In [20]:
# vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
# Learn vocab and transform data into form we want

    # Started at 7:09 PM / 11-24-19
    # data_1 has 3,372,904 rows

# sparse = tfidf.fit_transform(data_1)

# Print word counts
    # print('Vocabulary: ')
    #print(vect.vocabulary_)
    
# Get feature names to use as dataframe column headers

dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame

#dtm.head(50)

# Testing By Breaking Up Data

In [26]:
# Attempting to Break Up Data
    # Breaking up data column into TWO Columns. 
        # data_1 & data_2 
        # data_1 will have 3,372,904 rows. 
        # data_2 will have 3,372,905 rows. 
#
#print(data.shape)
#data_1 = data[0:1686452]
#data_2 = data[1686452]
#print(data_1)

(6745808,)


In [ ]:
# Testing by calling fit and transform separately on data_1
# Using %%timeit to see execution time. 
#%timeit
# Started at 6:49 PM / 11-25-19
#fitted_V = tfidf.fit(data_1)

# Testing Something New: ***THE RIGHT ONE***

In [21]:
# Checking for Missing Values in DataFrame
df.isna().sum()

stars          131088
text           131090
date           190999
total_votes    191022
tokens         191022
dtype: int64

In [25]:
# Dropping Missing Values 
df = df.dropna()

# Checking to see if it worked.
print(df.isna().sum())

# Checking Shape
df.shape

stars          0
text           0
date           0
total_votes    0
tokens         0
dtype: int64


(6685874, 5)

In [ ]:
# Since we already have the tokens in the DF,
# we can possibly fit it directly and bypass stop word/punct filtering. 

### WORKED / 1:40 PM / 11-28-2019 ###
### 10,0000 Rows Passed. Pickled as vect_1 ###

# vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

### TRAINED MODEL ####
    # vect = tfidf.fit(df.tokens[0:10000])
#### Completed at 1:40 PM / 11-28-2019 ####

# Training Another:  V2
    # Passing 100,000 Rows.
# vect2 = tfidf.fit(df.tokens[0:100000])

# Training Another. V3 
    # Passing 6,685,874 rows of df.tokens column through Vectorizer 
    # Lol, Didn't Work. 
#vect3 = tfidf.fit(df.tokens[0:6685874]) 

In [23]:
# Importing PrettyPrint to view 
from pprintpp import pprint as pp

# Checking to see if it worked....
# Print word counts

# pp('Vocabulary: ')
# pp(vect.vocabulary_)

In [27]:
# Pickling the Model
from sklearn.externals import joblib
joblib.dump(vect, 'vect_1.sav')

['vect_1.sav']

In [30]:
# import model
vect = joblib.load('vect_1.sav')

In [ ]:
dtm = pd.DataFrame(vect.todense(), columns=tfidf.get_feature_names())

In [ ]:
# Updated

# Testing Without Running Tokenzier.
The Dataframe already has the Tokens. No Need to Tokenize Twice. 
WIll be Running 

In [12]:
# Read-in df.csv
df = pd.read_csv('df.csv')
# Dropping Column
df = df.drop(columns=['Unnamed: 0'])
# Dropping all Missing / Na Values from Entire Dataframe
df = df.dropna()

In [37]:
#data = df['text']
# Checking Null Values and Shape
print(df.isna().sum())
print(df.shape)

stars          0
text           0
date           0
total_votes    0
tokens         0
dtype: int64
(6685874, 5)


In [38]:
df.isnull().sum()

stars          0
text           0
date           0
total_votes    0
tokens         0
dtype: int64

In [30]:
data = df['text']

In [36]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object -- also setting max and min on df etc...
tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
# Learn vocab and transform data into form we want
 # Passing 150,000 Rows of df['text'] through Vectorizer.
    
sparse = tfidf.fit_transform(df.text[0:150000])

TypeError: decoding to str: need a bytes-like object, Series found

In [33]:
# Create a vocabulary and get word counts per document
    # Learn vocab and transform data into form we want
 # Passing 100,000 Rows of df['text'] through Vectorizer.   
#vect = tfidf.fit_transform(df['text'])

TypeError: decoding to str: need a bytes-like object, Series found

## Testing a Progress Bar install for Vectorizer

In [ ]:
# Testing a Progress Bar Install for the Vectorizer.
# Works pretty quickly..
# Given that nothing is running.. HA! 
# Also a pointless feature that I spent 20 minutes on. . . 

#import pyprind
#import psutil
#import random
#import time

#n = 100
#bar = pyprind.ProgBar(n, monitor=True)
#for i in range(n):
    #time.sleep(0.05)
#tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))
    #bar.update()
#print(bar)